In [30]:
!pip install pdf2image
!pip install img2pdf
!apt-get install poppler-utils

from pathlib import Path
from pdf2image import convert_from_path
import img2pdf
from PIL import Image 
from google.colab import files
import cv2
import os
import numpy as np
from natsort import natsorted
import shutil

Reading package lists... Done
Building dependency tree       
Reading state information... Done
poppler-utils is already the newest version (0.62.0-2ubuntu2.12).
0 upgraded, 0 newly installed, 0 to remove and 37 not upgraded.


In [ ]:
file = files.upload() #pdfアップロード


In [25]:
base_path = "/content"
# PDFファイル名.pdf
pdf_filename = next(iter(file))
#画像の保存先
output_dir = Path("/content/pdfdivider/images")
# 出力するPDFの名前(/content/outpit_元ファイル名.pdf)
output_filename = os.path.join(base_path,('output_'+pdf_filename))

In [28]:
#画像を保存するディレクトリ作成
output_dir.mkdir(exist_ok=True)

#pdfをndarrayに変換
images = list(map(np.asarray, convert_from_path(os.path.join(base_path, pdf_filename),fmt='jpeg',output_file=os.path.join(base_path, pdf_filename))))

#画像を分割して保存
for j,image in enumerate(images): 
  im_rgb = cv2.cvtColor(image, cv2.COLOR_BGR2RGB) #RGに変換
  rows = 3  # 行数
  cols = 3  # 列数
  chunks = []
  for row_img in np.array_split(im_rgb, rows, axis=0):
      for chunk in np.array_split(row_img, cols, axis=1):
          chunks.append(chunk)
  for i, chunk in enumerate(chunks):
    save_path = output_dir / f"chunk{j:02d}_{i:02d}.jpeg"
    cv2.imwrite(str(save_path), chunk)

In [31]:
with open(output_filename,"wb") as f:
  # 画像をpdfに書き出し
  f.write(img2pdf.convert([Image.open(str(output_dir / j)).filename for j in natsorted(os.listdir(output_dir))]))

In [26]:
shutil.rmtree(output_dir) #画像を一括削除